In [59]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import datetime
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from tqdm import tqdm

# examples partially taken from https://nlpforhackers.io/complete-guide-to-spacy/
import spacy
# nlp = spacy.load('en_core_web_sm')
import en_core_web_sm
nlp = en_core_web_sm.load()

import warnings
warnings.filterwarnings("ignore")

In [83]:
data = pd.read_csv('data/cleaned.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263499 entries, 0 to 263498
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 197962 non-null  object 
 1   text               197963 non-null  object 
 2   create_at          157508 non-null  object 
 3   geo                101815 non-null  object 
 4   retweet_count      157508 non-null  object 
 5   like_count         157508 non-null  object 
 6   hashtags           157508 non-null  object 
 7   username           157508 non-null  object 
 8   following          157508 non-null  float64
 9   followers          157508 non-null  object 
 10  user_total_tweets  157507 non-null  float64
 11  user_likes_count   157507 non-null  object 
 12  cleaned_text       157506 non-null  object 
dtypes: float64(2), object(11)
memory usage: 26.1+ MB


In [84]:
data.columns

Index(['id', 'text', 'create_at', 'geo', 'retweet_count', 'like_count',
       'hashtags', 'username', 'following', 'followers', 'user_total_tweets',
       'user_likes_count', 'cleaned_text'],
      dtype='object')

In [85]:
data = data[(data['hashtags']!='ShopParaTi') & (data['hashtags']!='22.0')]
data['hashtags'].value_counts()

#ukraine          27000
#covid            27000
#christmas        26690
#vegan            26651
#climatechange    26089
#ftx              24076
Name: hashtags, dtype: int64

In [86]:
data = data[data['cleaned_text'].notna()]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157506 entries, 0 to 263498
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 157506 non-null  object 
 1   text               157506 non-null  object 
 2   create_at          157506 non-null  object 
 3   geo                101812 non-null  object 
 4   retweet_count      157506 non-null  object 
 5   like_count         157506 non-null  object 
 6   hashtags           157506 non-null  object 
 7   username           157506 non-null  object 
 8   following          157506 non-null  float64
 9   followers          157506 non-null  object 
 10  user_total_tweets  157506 non-null  float64
 11  user_likes_count   157506 non-null  object 
 12  cleaned_text       157506 non-null  object 
dtypes: float64(2), object(11)
memory usage: 16.8+ MB


In [71]:
num_noun = []
num_verb = []
num_adj = []
num_adv = []
total_length = []
for n in tqdm(data['text']):
    doc = nlp(n)
    noun, verb, adj, adv, length = 0,0,0,0,0
    for i, sent in enumerate(doc.sents):
        for token in sent:
            length += 1
            if token.pos_ == 'NOUN': noun += 1
            elif token.pos_ == 'VERB': verb += 1
            elif  token.pos_ == 'ADJ': adj += 1
            elif  token.pos_ == 'ADV': adv += 1
    num_noun.append(noun)
    num_verb.append(verb)
    num_adj.append(adj)
    num_adv.append(adv)
    total_length.append(length)

100%|██████████| 157506/157506 [19:57<00:00, 131.49it/s]


In [97]:
data['num_noun']=num_noun
data['num_verb']=num_verb
data['num_adj']=num_adj
data['num_adv']=num_adv
data['total_length']=total_length
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157506 entries, 0 to 263498
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 157506 non-null  object 
 1   text               157506 non-null  object 
 2   create_at          157506 non-null  object 
 3   geo                101812 non-null  object 
 4   retweet_count      157506 non-null  float64
 5   like_count         157506 non-null  float64
 6   hashtags           157506 non-null  object 
 7   username           157506 non-null  object 
 8   following          157506 non-null  float64
 9   followers          157506 non-null  float64
 10  user_total_tweets  157506 non-null  float64
 11  user_likes_count   157506 non-null  float64
 12  cleaned_text       157506 non-null  object 
 13  num_noun           157506 non-null  int64  
 14  num_vern           157506 non-null  int64  
 15  num_adj            157506 non-null  int64  
 16  nu

In [102]:
data = pd.get_dummies(data=data, columns=['hashtags'])
data.columns

Index(['id', 'text', 'create_at', 'geo', 'retweet_count', 'like_count',
       'username', 'following', 'followers', 'user_total_tweets',
       'user_likes_count', 'cleaned_text', 'num_noun', 'num_vern', 'num_adj',
       'num_adv', 'total_length', 'num_verb', 'hashtags_#christmas',
       'hashtags_#climatechange', 'hashtags_#covid', 'hashtags_#ftx',
       'hashtags_#ukraine', 'hashtags_#vegan'],
      dtype='object')

In [103]:
data[['retweet_count','like_count','following','followers','user_total_tweets','user_likes_count']] = data[['retweet_count','like_count','following','followers','user_total_tweets','user_likes_count']].astype('float')
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 157506 entries, 0 to 263498
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       157506 non-null  object 
 1   text                     157506 non-null  object 
 2   create_at                157506 non-null  object 
 3   geo                      101812 non-null  object 
 4   retweet_count            157506 non-null  float64
 5   like_count               157506 non-null  float64
 6   username                 157506 non-null  object 
 7   following                157506 non-null  float64
 8   followers                157506 non-null  float64
 9   user_total_tweets        157506 non-null  float64
 10  user_likes_count         157506 non-null  float64
 11  cleaned_text             157506 non-null  object 
 12  num_noun                 157506 non-null  int64  
 13  num_vern                 157506 non-null  int64  
 14  num_

In [120]:
data['like_count(log)']=  np.log(data['like_count'])
data['retweet_count(log)']=  np.log(data['retweet_count'])

In [107]:
data.to_csv('data_with_meta.csv',header=None)

In [153]:
useful_columns = [
    'following',
    'followers', 
    'user_total_tweets',
    'user_likes_count',
    'hashtags_#christmas', 
    'hashtags_#climatechange',
    'hashtags_#covid', 
    'hashtags_#ftx', 
    'hashtags_#ukraine',
    'hashtags_#vegan',
    'retweet_count', 
    'like_count',
    'retweet_count(log)',
    'like_count(log)',
    'num_noun',
    'num_verb',
    'num_adj',
    'num_adv',
    'total_length'
]

baseline_features = [
    'following',
    'followers', 
    'user_total_tweets',
    'user_likes_count',
    'hashtags_#christmas', 
    'hashtags_#climatechange',
    'hashtags_#covid', 
    'hashtags_#ftx', 
    'hashtags_#ukraine',
    'hashtags_#vegan',
]


all_features = [
    'following',
    'followers', 
    'user_total_tweets',
    'user_likes_count',
    'hashtags_#christmas', 
    'hashtags_#climatechange',
    'hashtags_#covid', 
    'hashtags_#ftx', 
    'hashtags_#ukraine',
    'hashtags_#vegan',
    'num_noun',
    'num_verb',
    'num_adj',
    'num_adv',
    'total_length'
]
data_metadata = data[useful_columns]
data_metadata = data_metadata[data_metadata['like_count']!=0]
# data_metadata = data_metadata[data_metadata['retweet_count']!=0]
data_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23308 entries, 0 to 263489
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   following                23308 non-null  float64
 1   followers                23308 non-null  float64
 2   user_total_tweets        23308 non-null  float64
 3   user_likes_count         23308 non-null  float64
 4   hashtags_#christmas      23308 non-null  uint8  
 5   hashtags_#climatechange  23308 non-null  uint8  
 6   hashtags_#covid          23308 non-null  uint8  
 7   hashtags_#ftx            23308 non-null  uint8  
 8   hashtags_#ukraine        23308 non-null  uint8  
 9   hashtags_#vegan          23308 non-null  uint8  
 10  retweet_count            23308 non-null  float64
 11  like_count               23308 non-null  float64
 12  retweet_count(log)       23308 non-null  float64
 13  like_count(log)          23308 non-null  float64
 14  num_noun             

In [154]:
def knn(X_train, X_dev, y_train, y_dev):
    knn = KNeighborsRegressor(n_neighbors=5)
    knn.fit(X_train, y_train)
    prediction = knn.predict(X_dev)
    knn_mse = mean_squared_error(y_dev, prediction)
    knn_r2 = r2_score(y_dev, prediction)
    return knn_mse, knn_r2


def svr(X_train, X_dev, y_train, y_dev):
    svr = SVR()
    svr.fit(X_train, y_train)
    prediction = svr.predict(X_dev)
    svc_mse = mean_squared_error(y_dev, prediction)
    svr_r2 = r2_score(y_dev, prediction)
    return svc_mse, svr_r2

# XGBRegressor
def xgbr(X_train, X_dev, y_train, y_dev):
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', random_state=0, n_estimators = 50)
    xg_reg.fit(X_train,y_train)
    xg_preds = xg_reg.predict(X_dev)
    xg_mse = mean_squared_error(y_dev, xg_preds)
    xg_r2 = r2_score(y_dev, xg_preds)
    
#     # Mean Squared Error
#     xg_mse = mean_squared_error(np.e ** y_dev, np.e ** xg_preds)

#     # r2 score
#     xg_r2 = r2_score(np.e ** y_dev, np.e ** xg_preds)

    return xg_mse, xg_r2

# GradientBoostingRegressor
def gbr(X_train, X_dev, y_train, y_dev):
    gra_reg = GradientBoostingRegressor(random_state=0, n_estimators = 50)
    gra_reg.fit(X_train,y_train)
    gra_preds = gra_reg.predict(X_dev)
    gra_mse = mean_squared_error(y_dev, gra_preds)
    gra_r2 = r2_score(y_dev, gra_preds)
    return gra_mse, gra_r2

# AdaBoostRegressor
def ada(X_train, X_dev, y_train, y_dev):
    ada_reg = AdaBoostRegressor(random_state=0, n_estimators = 50)
    ada_reg.fit(X_train,y_train)
    ada_preds = ada_reg.predict(X_dev)
    ada_mse = mean_squared_error(y_dev, ada_preds)
    ada_r2 = r2_score(y_dev, ada_preds)
    return ada_mse, ada_r2

def lasso(X_train, X_dev, y_train, y_dev):
   
    lasso = Lasso()
    lasso.fit(X_train, y_train)
    prediction = lasso.predict(X_dev)
    lasso_mse = mean_squared_error(y_dev.values, prediction)
    lasso_r2 = r2_score(y_dev, prediction)
    return lasso_mse, lasso_r2

def ridge(X_train, X_dev, y_train, y_dev):
   
    ridge = Ridge()
    ridge.fit(X_train, y_train)
    prediction = ridge.predict(X_dev)
    ridge_mse = mean_squared_error(y_dev, prediction)
    ridge_r2 = r2_score(y_dev, prediction)
    return ridge_mse, ridge_r2


# Linear regression
def linreg(X_train, X_dev, y_train, y_dev):
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    prediction = linreg.predict(X_dev)
    linreg_mse = mean_squared_error(y_dev, prediction)
    linreg_r2 = r2_score(y_dev, prediction)
    
    # Mean Squared Error
    lm_mse = mean_squared_error(np.e ** y_dev, np.e ** prediction)

    # r2 score
    lm_r2 = r2_score(np.e ** y_dev, np.e ** prediction)
    
#     return linreg_mse, linreg_r2
    return lm_mse, lm_r2


# Random Forests
def rf(X_train, X_dev, y_train, y_dev):
    rf = RandomForestRegressor(random_state=0)
    rf.fit(X_train,y_train)
    prediction = rf.predict(X_dev)
    rf_mse = mean_squared_error(y_dev, prediction)
    rf_r2 = r2_score(y_dev, prediction)
    
    
    # Mean Squared Error
#     rf_mse = mean_squared_error(np.e ** y_dev, np.e ** prediction)

#     # r2 score
#     rf_r2 = r2_score(np.e ** y_dev, np.e ** prediction)
    
    return rf_mse, rf_r2
    

In [157]:
features = data_metadata[all_features]
labels = data_metadata['like_count']
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

    
rf = RandomForestRegressor(random_state=0)
rf.fit(X_train,y_train)
prediction = rf.predict(X_test)
rf_mse = mean_squared_error(y_test, prediction)
rf_r2 = r2_score(y_test, prediction)
mean_squared_log_error(y_test, rf.predict(X_test))


1.1462728778077955

In [158]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
mean_squared_log_error(y_test, knn.predict(X_test))

0.9920569930873737

In [159]:
from sklearn.metrics import mean_squared_log_error
features = data_metadata[baseline_features]
labels = data_metadata['like_count']
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


    
rf = RandomForestRegressor(random_state=0)
rf.fit(X_train,y_train)
prediction = rf.predict(X_test)
mean_squared_log_error(y_test, rf.predict(X_test))



1.0079768105438907

In [160]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
mean_squared_log_error(y_test, knn.predict(X_test))

0.986771649360644